## GENERACIÓN DEL GRAFO RDF

In [3]:
!pip install rdflib

  Obtaining dependency information for rdflib from https://files.pythonhosted.org/packages/b9/20/35d2baebacf357b562bd081936b66cd845775442973cb033a377fd639a84/rdflib-7.5.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 6.6 MB/s eta 0:00:0000:0100:01


In [13]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

In [14]:
# CARGAR DEL DATASTES FINAL DE INCENDIOS
df = pd.read_csv(
    "/Users/irenealavesruiz/Downloads/Dataset_Maestro_FINAL_DEF_V3.csv",
    sep=";",
    encoding="latin1"
)

df = df.reset_index(drop=True)

print("Filas cargadas:", len(df))
df.head()

Filas cargadas: 49982


,numeroparte,IdPif,IdParteMonte,medios_aereos_desplegados,medios_personal_desplegados,superficie_prote_total,t_deteccion,t_extincion,superficie_total_ha,id_agente_causante,...,Es_Grande,id_dim_espacio,id_tipo_vegetacion,especie,cod_modelo_combustion,id_localizacion,Comunidad,Provincia,Municipio,Comarca
0,2015010007,1141741,1,0,6,0.00,10,45,1.0,1,...,0.0,-1.0,74,ROBINIA ...,SIN_DATOS ...,2835,EUSKADI,ALAVA,Municipio 10,CANTABRICA
1,2015010011,1141745,1,0,4,0.00,10,35,5.0,1,...,0.0,-1.0,82,CELTIS ...,SIN_DATOS ...,3792,EUSKADI,ALAVA,Municipio 20,RIBERAS
2,2015010012,1141746,1,0,4,0.00,6,33,2.0,1,...,0.0,-1.0,-1,Especie Desconocida,Modelo Desconocido,3792,EUSKADI,ALAVA,Municipio 20,RIBERAS
3,2015010016,1141750,1,0,3,0.00,80,300,2.0,1,...,0.0,-1.0,30,QUERCUS ...,SIN_DATOS ...,2855,EUSKADI,ALAVA,Municipio 4,CANTABRICA
4,2015020084,1141789,1,2,16,0.24,27,347,24.0,1,...,1.0,854.0,-1,Especie Desconocida,Modelo Desconocido,55,CASTILLA-MANCHA,ALBACETE,Municipio 37,HELLIN


In [15]:
import unicodedata
import pandas as pd

# Función de normalización para igualar nombres de provincias
def normalizar_provincia(p):
    if pd.isna(p):
        return "DESCONOCIDO"

    # Pasar a string y quitar acentos
    p = str(p).strip().upper()
    p = unicodedata.normalize('NFKD', p).encode('ascii', 'ignore').decode('utf-8')

    # Correcciones específicas necesarias para enlazar bien con Wikidata
    if p in ["ILLES BALEARS", "ILLAS BALEARS", "ISLAS BALEARES"]:
        return "BALEARES"
    if p in ["S.C. TENERIFE", "SC TENERIFE", "SANTA CRUZ TENERIFE", "TENERIFE"]:
        return "SANTA CRUZ DE TENERIFE"
    if p == "SANTANDER":
        return "CANTABRIA"
    if p in ["A CORUNA", "LA CORUNA"]:
        return "A CORUNA"

    return p

# Aplicar normalización
df["Provincia"] = df["Provincia"].apply(normalizar_provincia)

# Verificar provincias normalizadas
print("Provincias únicas normalizadas:")
sorted(df["Provincia"].unique())

Provincias únicas normalizadas:


['A CORUNA',
 'ALAVA',
 'ALBACETE',
 'ALICANTE',
 'ALMERIA',
 'ASTURIAS',
 'AVILA',
 'BADAJOZ',
 'BALEARES',
 'BARCELONA',
 'BURGOS',
 'CACERES',
 'CADIZ',
 'CANTABRIA',
 'CASTELLON',
 'CEUTA',
 'CIUDAD REAL',
 'CORDOBA',
 'CUENCA',
 'GIRONA',
 'GRANADA',
 'GUADALAJARA',
 'GUIPUZCOA',
 'HUELVA',
 'HUESCA',
 'JAEN',
 'LA RIOJA',
 'LAS PALMAS',
 'LEON',
 'LLEIDA',
 'LUGO',
 'MADRID',
 'MALAGA',
 'MURCIA',
 'NAVARRA',
 'OURENSE',
 'PALENCIA',
 'PONTEVEDRA',
 'PROVINCIA DESCONOCIDA',
 'SALAMANCA',
 'SANTA CRUZ DE TENERIFE',
 'SEGOVIA',
 'SEVILLA',
 'SORIA',
 'TARRAGONA',
 'TERUEL',
 'TOLEDO',
 'VALENCIA',
 'VALLADOLID',
 'VIZCAYA',
 'ZAMORA',
 'ZARAGOZA']

In [16]:
# DEFINIR NAMESPACES
SCHEMA = Namespace("https://schema.org/")
EX = Namespace("http://example.org/")

In [17]:
# CREAR GRAFO RDF
g = Graph()
g.bind("schema", SCHEMA)
g.bind("ex", EX)

In [18]:
wikidata_provincias = {
    "ALAVA": "Q81801",
    "ALBACETE": "Q12224",
    "ALICANTE": "Q54936",
    "ALMERIA": "Q12233",
    "ASTURIAS": "Q3946",
    "AVILA": "Q12245",
    "BADAJOZ": "Q12250",
    "BALEARES": "Q1764",
    "BARCELONA": "Q1492",
    "BURGOS": "Q12104",
    "CACERES": "Q12257",
    "CADIZ": "Q12260",
    "CANTABRIA": "Q3890",
    "CASTELLON": "Q12268",
    "CIUDAD REAL": "Q12274",
    "CORDOBA": "Q12278",
    "CUENCA": "Q12283",
    "GIRONA": "Q7194", 
    "GRANADA": "Q12290",
    "GUADALAJARA": "Q12293",
    "GUIPUZCOA": "Q95010",
    "HUELVA": "Q12299",
    "HUESCA": "Q12303",
    "JAEN": "Q12306",
    "LEON": "Q12309",
    "LLEIDA": "Q12314",
    "LA RIOJA": "Q12329",
    "LUGO": "Q12319",
    "MADRID": "Q2807",
    "MALAGA": "Q12321",
    "MURCIA": "Q12325",
    "NAVARRA": "Q10346",
    "OURENSE": "Q12366",
    "PALENCIA": "Q12362",
    "PONTEVEDRA": "Q12359",
    "SALAMANCA": "Q12373",
    "SANTA CRUZ DE TENERIFE": "Q12703",
    "SEGOVIA": "Q12377",
    "SEVILLA": "Q12382",
    "SORIA": "Q12386",
    "TARRAGONA": "Q12391",
    "TERUEL": "Q12394",
    "TOLEDO": "Q12401",
    "VALENCIA": "Q12389",
    "VALLADOLID": "Q12398",
    "VIZCAYA": "Q5768",
    "ZAMORA": "Q12408",
    "ZARAGOZA": "Q12289",
    "A CORUNA": "Q16552",
    "LAS PALMAS": "Q12709",
    "CEUTA": "Q24767",
    "MELILLA": "Q24672"
}


# TRANSFORMACIÓN A RDF
for row in df.itertuples():

    # ----- PLACE (Provincia) -----
    prov_name = str(row.Provincia).strip()
    prov_id = prov_name.replace(" ", "_").upper()
    prov_uri = EX[f"place/{prov_id}"]

    # Crear el recurso Place
    g.add((prov_uri, RDF.type, SCHEMA.Place))
    g.add((prov_uri, SCHEMA.name, Literal(prov_name)))

    # Enlace a Wikidata (si existe)
    prov_key = prov_name.upper()
    if prov_key in wikidata_provincias:
        wd_id = wikidata_provincias[prov_key]
        g.add((prov_uri,
               SCHEMA.sameAs,
               URIRef(f"https://www.wikidata.org/entity/{wd_id}")))

    # ----- EVENT (Incendio) -----
    event_id = str(row.numeroparte)
    event_uri = EX[f"event/{event_id}"]

    g.add((event_uri, RDF.type, SCHEMA.Event))
    g.add((event_uri, SCHEMA.name, Literal("Incendio forestal")))

    # Relación con el lugar
    g.add((event_uri, SCHEMA.location, prov_uri))

    # Año del incendio
    g.add((event_uri, SCHEMA.startDate,
           Literal(str(row.Anio), datatype=XSD.gYear)))

    # Causa del incendio
    causa = str(row.descripcion_agente).strip()
    g.add((event_uri, SCHEMA.eventType, Literal(causa)))

In [19]:
# GUARDAT ARCHIVO RDF
output_path = "/Users/irenealavesruiz/Downloads/incendios.ttl"
g.serialize(output_path, format="turtle")
print("Archivo RDF generado en:", output_path)

Archivo RDF generado en: /Users/irenealavesruiz/Downloads/incendios.ttl


In [20]:
print("=======================================")
print(" COMPROBACIONES DEL GRAFO RDF GENERADO")
print("=======================================\n")

# Número total de tripletas
num_triples = len(g)
print(f"Total de tripletas en el grafo: {num_triples}")

# Contar eventos (schema:Event)
num_eventos = len(list(g.subjects(RDF.type, SCHEMA.Event)))
print(f"Número total de eventos (schema:Event): {num_eventos}")

# Contar lugares (schema:Place)
num_lugares = len(list(g.subjects(RDF.type, SCHEMA.Place)))
print(f"Número total de lugares (schema:Place): {num_lugares}")

# Mostrar 5 primeras tripletas como ejemplo
print("\nEjemplos de tripletas (primeras 5):\n")
for i, triple in enumerate(g, start=1):
    print(triple)
    if i == 5:
        break

# Mostrar estructura final del TTL (10 primeras líneas)
print("\nEstructura del archivo TTL (primeras líneas):")
with open("/Users/irenealavesruiz/Downloads/incendios.ttl") as f:
    for i in range(8):
        print(f.readline().strip())

 COMPROBACIONES DEL GRAFO RDF GENERADO

Total de tripletas en el grafo: 193957
Número total de eventos (schema:Event): 38759
Número total de lugares (schema:Place): 52

Ejemplos de tripletas (primeras 5):

(rdflib.term.URIRef('http://example.org/event/2017330160'), rdflib.term.URIRef('https://schema.org/eventType'), rdflib.term.Literal('Desconocida'))
(rdflib.term.URIRef('http://example.org/event/2015180068'), rdflib.term.URIRef('https://schema.org/name'), rdflib.term.Literal('Incendio forestal'))
(rdflib.term.URIRef('http://example.org/event/2015150064'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://schema.org/Event'))
(rdflib.term.URIRef('http://example.org/event/2019331200'), rdflib.term.URIRef('https://schema.org/location'), rdflib.term.URIRef('http://example.org/place/ASTURIAS'))
(rdflib.term.URIRef('http://example.org/event/2018360037'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef(